# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [64]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification is GNU 7.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/martin/.pyenv/shims/python (found version "3.6.5") 
-- Found PythonLibs: /home/martin/.pyenv/versions/3.6.5/lib/libpython3.6m.a
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/martin/Desktop/repos/metnum-tp2-20191c
-- Configuring 

In [65]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/home/martin/Desktop/repos/metnum-tp2-20191c/notebooks
Python 3.6.5


In [66]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
./._imdb_small.csv
imdb_small.csv
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,There are many different versions of this one ...,neg,6626_7.txt
freq,6275,2,6322,2


In [67]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [6]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [31]:
import sentiment

clf = sentiment.KNNClassifier(100)
#esta linea y el siguiente bloque son para correr KNN sin PCA
#esta comentado para poder ejecutar todos los bloques sin ejecutar este
#por equivocacion
#clf.fit(X_train, y_train)


In [32]:
#%%time
#from sklearn.metrics import accuracy_score
#y_pred = clf.predict(X_test)

#acc = accuracy_score(y_test, y_pred)
#print("Accuracy: {}".format(acc))

In [33]:
pca = sentiment.PCA(30)

In [34]:
pca.fit(X_train,10000, 0.000000001)

In [35]:
pca_X_train = pca.transform(X_train, 5)
pca_X_test = pca.transform(X_test,5)

In [36]:
print(pca_X_test.shape[1])
print(pca_X_test)

5
[[ 4.04970925e+00  8.29573503e-01  8.53913596e-01  2.05502313e-01
  -1.82184614e-01]
 [ 4.64798799e+00  8.56768700e-01 -1.41044555e+00 -3.67284553e-01
  -1.13595151e+00]
 [ 1.63599108e+00  9.32925907e-03  1.21861747e-01  1.04115637e-01
  -3.46904729e-01]
 ...
 [ 1.05671920e+01  1.80047083e+00 -1.88015752e+00 -8.40803435e-01
   3.11108248e-01]
 [ 3.18381510e+00  4.49905049e-01 -4.37183546e-01 -3.76908099e-01
  -3.24407551e-01]
 [ 9.82251282e+00  2.72646357e+00 -9.34823084e-01  7.58754122e-01
  -1.41311295e+00]]


In [74]:
%%time
clf.fit(pca_X_train, y_train)
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

y_pred = clf.predict(pca_X_test)

acc = accuracy_score(y_test, y_pred)
pre = accuracy_score(y_test, y_pred)
rec = accuracy_score(y_test, y_pred)


print("Accuracy: {}".format(acc))
print("Precision: {}".format(pre))
print("Recall: {}".format(rec))


Accuracy: 0.5263745019920318
Precision: 0.5263745019920318
Recall: 0.5263745019920318
CPU times: user 3.99 s, sys: 19.9 ms, total: 4.01 s
Wall time: 3.99 s


In [75]:
#All in one. Thuis cell defines the tester functions.
import time
from sklearn.feature_extraction.text import CountVectorizer
def test(max_df_=0.90, min_df_=0.01, max_features_=5000, knn_neighbours_=100, alpha_=30, log=False):

    vectorizer = CountVectorizer(max_df=max_df_, min_df=min_df_, max_features=max_features_)
    vectorizer.fit(text_train)

    X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
    X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

    import sentiment

    clf = sentiment.KNNClassifier(knn_neighbours_)
    #esta linea y el siguiente bloque son para correr KNN sin PCA
    #esta comentado para poder ejecutar todos los bloques sin ejecutar este
    #por equivocacion
    #clf.fit(X_train, y_train)

    pca = sentiment.PCA(alpha_)
    pca.fit(X_train,10000, 0.000000001)
    pca_X_train = pca.transform(X_train, 5)
    pca_X_test = pca.transform(X_test,5)

    #Timing starts, same as previous cell:
    start = time.time()

    clf.fit(pca_X_train, y_train)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score

    y_pred = clf.predict(pca_X_test)

    acc = accuracy_score(y_test, y_pred)
    pre = accuracy_score(y_test, y_pred)
    rec = accuracy_score(y_test, y_pred)

    #Timing ends, again, like in previous cell.
    end = time.time()
    delta_time = end - start
    delta_time_str = str(round(delta_time, 2))
    if log:
        comma = ", "
        f = open("../data/test_results/out.csv","a+")
        f.write("\n")
        f.write(str(round(acc,2)) + comma)
        f.write(str(round(pre,2)) + comma)
        f.write(str(round(rec,2)) + comma)
        f.write(delta_time_str + comma)
        f.write(str(round(max_df_,2)) + comma)
        f.write(str(round(min_df_,2)) + comma)
        f.write(str(round(max_features_,2)) + comma)
        f.write(str(round(knn_neighbours_,2)) + comma)
        f.write(str(round(alpha_,2)))
        f.close()

    else:
        print("Time:" + delta_time_str)
        print("Accuracy: {}".format(acc))
        print("Precision: {}".format(pre))
        print("Recall: {}".format(rec))
    


In [97]:
#Actual testing, this cell runs the tester functions and logs or prints results.
#It may take a long time to run.
import numpy as np
test_individually = False

if test_individually:
    test(max_df_=0.99, min_df_=0.1, max_features_=5000, knn_neighbours_=100, alpha_=30, log=True)
else:
    for max_df_ in np.arange(0.98, 0.99, 0.01):
        for min_df_ in np.arange(0.1, 0.2, 0.1):
            for max_features_ in range(5000, 5001, 1):
                for knn_neighbours_ in range(5, 151, 5):
                    for alpha_ in range(1, 50, 1):
                        test(max_df_, min_df_, max_features_, knn_neighbours_, alpha_, log=True)   



KeyboardInterrupt: 

[0.1 0.2]
